In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import joblib

# Veriyi yükle
df = pd.read_csv('data_vectorized.csv')

# Özellikler (X) ve Etiketler (y) ayırma
X = df.drop(columns=['Label'])  # Özellikler (vectorized data)
y = df['Label']  # Etiketler (True/False)

# Etiketleri sayılara dönüştürme (Label Encoding)
le = LabelEncoder()
y = le.fit_transform(y)

# Eğitim ve test verilerini ayırma (%80 eğitim, %20 test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Parametreleri
embedding_dim = 100  # Embedding boyutu
max_len = X.shape[1]  # Veri uzunluğu (her kelimenin vektör boyutu)
num_classes = len(np.unique(y))  # Sınıf sayısı (True/False)

# LSTM Modeli oluşturma
model = Sequential([
    Embedding(input_dim=len(X.columns) + 1,  # Girdi boyutu, toplam kelime sayısı
              output_dim=embedding_dim,  # Embedding vektör boyutu
              input_length=max_len),  # Girdi uzunluğu
    Dropout(0.2),  # Aşırı öğrenmeyi engellemek için dropout
    LSTM(128, activation='tanh'),  # LSTM katmanı
    Dense(64, activation='relu'),  # Tam bağlı katman
    Dense(num_classes, activation='softmax')  # Çıkış katmanı (binary classification için softmax)
])

# Modeli derleme
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Modelin özetini görüntüleme
model.summary()

# Modeli eğitme
history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test))

# Eğitim ve doğrulama doğruluğunu görselleştirme
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Model Doğruluğu')
plt.xlabel('Epok')
plt.ylabel('Doğruluk')
plt.legend(loc='best')
plt.show()

# Test verisi üzerinde tahmin yapma
y_pred = model.predict(X_test)
y_pred_class = np.argmax(y_pred, axis=1)  # Softmax çıktısından en yüksek sınıfı seçiyoruz

# Accuracy
accuracy = accuracy_score(y_test, y_pred_class)
print(f'Accuracy: {accuracy:.4f}')

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_class)
print('Confusion Matrix:')
print(cm)

# Classification Report (Precision, Recall, F1-score)
class_report = classification_report(y_test, y_pred_class)
print('Classification Report:')
print(class_report)

# ROC-AUC
roc_auc = roc_auc_score(y_test, model.predict(X_test)[:, 1])  # Binary classification için
print(f'ROC-AUC: {roc_auc:.4f}')

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, model.predict(X_test)[:, 1])
plt.figure()
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


